In [1]:
!pip show tensorflow keras

Name: tensorflow
Version: 2.17.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: absl-py, astunparse, flatbuffers, gast, google-pasta, grpcio, h5py, keras, libclang, ml-dtypes, numpy, opt-einsum, packaging, protobuf, requests, setuptools, six, tensorboard, tensorflow-io-gcs-filesystem, termcolor, typing-extensions, wrapt
Required-by: dopamine_rl, tf_keras
---
Name: keras
Version: 3.4.1
Summary: Multi-backend Keras.
Home-page: https://github.com/keras-team/keras
Author: Keras team
Author-email: keras-users@googlegroups.com
License: Apache License 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: absl-py, h5py, ml-dtypes, namex, numpy, optree, packaging, rich
Required-by: tensorflow


In [2]:
# Step 1: Upload your kaggle.json file (API token)
from google.colab import files
files.upload()

# Step 2: Create a Kaggle directory and move the kaggle.json file to it
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# Step 3: Set permissions for the kaggle.json file
!chmod 600 ~/.kaggle/kaggle.json

# Step 4: Download the retinal disease classification dataset using the Kaggle API
!kaggle datasets download -d andrewmvd/retinal-disease-classification

# Step 5: Unzip the downloaded dataset
!unzip retinal-disease-classification.zip -d ./retinal_disease_classification_data/


Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/andrewmvd/retinal-disease-classification
License(s): other
100% 7.42G/7.43G [01:26<00:00, 162MB/s]
100% 7.43G/7.43G [01:26<00:00, 92.2MB/s]
Archive:  retinal-disease-classification.zip
  inflating: ./retinal_disease_classification_data/Evaluation_Set/Evaluation_Set/RFMiD_Validation_Labels.csv  
  inflating: ./retinal_disease_classification_data/Evaluation_Set/Evaluation_Set/Validation/1.png  
  inflating: ./retinal_disease_classification_data/Evaluation_Set/Evaluation_Set/Validation/10.png  
  inflating: ./retinal_disease_classification_data/Evaluation_Set/Evaluation_Set/Validation/100.png  
  inflating: ./retinal_disease_classification_data/Evaluation_Set/Evaluation_Set/Validation/101.png  
  inflating: ./retinal_disease_classification_data/Evaluation_Set/Evaluation_Set/Validation/102.png  
  inflating: ./retinal_disease_classification_data/Evaluation_Set/Evaluation_Set/Validation/103.png  
  inflating: ./r

In [3]:
# Step 6: Display the path of the dataset
import os
dataset_path = os.path.abspath("./retinal_disease_classification_data/")
print(f"Dataset extracted to: {dataset_path}")

train_dir = '/content/retinal_disease_classification_data/Training_Set/Training_Set/Training'

# Check contents of the directory
print(os.listdir(train_dir))


Dataset extracted to: /content/retinal_disease_classification_data
['618.png', '1712.png', '1094.png', '1820.png', '1418.png', '1629.png', '1914.png', '293.png', '685.png', '1042.png', '1737.png', '563.png', '499.png', '1185.png', '671.png', '1278.png', '1646.png', '262.png', '1343.png', '1868.png', '196.png', '427.png', '1651.png', '1256.png', '217.png', '736.png', '1437.png', '1519.png', '686.png', '1873.png', '394.png', '212.png', '1158.png', '1095.png', '1655.png', '1912.png', '1240.png', '963.png', '1518.png', '528.png', '160.png', '344.png', '195.png', '1811.png', '842.png', '234.png', '1870.png', '929.png', '715.png', '873.png', '676.png', '1423.png', '301.png', '959.png', '1520.png', '925.png', '986.png', '289.png', '57.png', '1568.png', '1181.png', '591.png', '1188.png', '699.png', '323.png', '94.png', '826.png', '1344.png', '1313.png', '570.png', '282.png', '1276.png', '1822.png', '730.png', '625.png', '333.png', '778.png', '1800.png', '1689.png', '825.png', '1896.png', '241.

In [6]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.metrics import r2_score, precision_recall_fscore_support, accuracy_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.callbacks import Callback

# Custom callback to track metrics
class MetricsCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        print(f"Epoch {epoch + 1}: Accuracy = {logs['accuracy']}, Loss = {logs['loss']}")

# Specify the directory containing images
image_dir = '/content/retinal_disease_classification_data/Evaluation_Set/Evaluation_Set/Validation/'

# List all files in the directory
image_files = os.listdir(image_dir)

# Initialize lists to hold images and corresponding health data
X = []
y = []

# Process each image
for img_file in image_files:
    image_path = os.path.join(image_dir, img_file)

    # Load the retinal image
    image = load_img(image_path, target_size=(224, 224))  # Resize each image
    image = img_to_array(image)
    image = image / 255.0  # Normalize pixel values

    # Append the image to the X list
    X.append(image)

    # Simulate health labels: [age, sbp, dbp, bmi, hba1c]
    # In practice, you would have these values from your dataset
    y.append([45, 130, 80, 26.5, 6.2])  # Example target labels

# Convert lists to numpy arrays
X = np.array(X)
y = np.array(y)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 2. Build the CNN model for feature extraction and prediction
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(5, activation='linear')  # Output: [age, sbp, dbp, bmi, hba1c]
])

# Compile the model with additional metrics
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae', 'accuracy'])

# 3. Train the model with a callback to track metrics
history = model.fit(X_train, y_train, epochs=10, validation_split=0.2, verbose=1, callbacks=[MetricsCallback()])

# 4. Make predictions
y_pred = model.predict(X_test)



/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.7523 - loss: 3303.9709 - mae: 43.8511Epoch 1: Accuracy = 0.8630806803703308, Loss = 2070.89501953125
13/13 ━━━━━━━━━━━━━━━━━━━━ 55s 4s/step - accuracy: 0.7602 - loss: 3215.8940 - mae: 43.1325 - val_accuracy: 1.0000 - val_loss: 657.6534 - val_mae: 18.1537
Epoch 2/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.9588 - loss: 769.6982 - mae: 21.6330Epoch 2: Accuracy = 0.980440080165863, Loss = 748.1406860351562
13/13 ━━━━━━━━━━━━━━━━━━━━ 52s 4s/step - accuracy: 0.9604 - loss: 768.1584 - mae: 21.6224 - val_accuracy: 1.0000 - val_loss: 362.5049 - val_mae: 13.8791
Epoch 3/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.9871 - loss: 595.8726 - mae: 19.0496Epoch 3: Accuracy = 0.9902200698852539, Loss = 619.2426147460938
13/13 ━━━━━━━━━━━━━━━━━━━━ 79s 4s/step - accuracy: 0.9874 - loss: 597.5419 - mae: 19.0624 - val_accuracy: 1.0000 - val_loss: 163.4018 - val_mae: 8.4878
Epoch 4/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step -

In [7]:
# Evaluate the model: R2 score, Precision, Recall, F1 Score, and Accuracy
# Evaluate the model: R2 score, MAE, and MSE
r2 = r2_score(y_test, y_pred)
mae = np.mean(np.abs(y_test - y_pred), axis=0)
mse = np.mean((y_test - y_pred) ** 2, axis=0)

print(f"R2 Score: {r2}")
print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")


# 5. Suggest health precautions based on the predictions
def suggest_precautions(y_pred):
    age, sbp, dbp, bmi, hba1c = y_pred[0]
    suggestions = []
    if sbp > 140 or dbp > 90:
        suggestions.append("Monitor blood pressure regularly and reduce salt intake.")
    if bmi > 25:
        suggestions.append("Maintain a healthy diet and engage in regular exercise.")
    if hba1c > 6.5:
        suggestions.append("Monitor blood sugar levels and consult a doctor.")
    if age > 50:
        suggestions.append("Consider regular heart check-ups.")
    return suggestions

# Generate suggestions based on predictions
precautions = suggest_precautions(y_pred)
print("Suggested Precautions: ", precautions)

# 6. Save the trained model in HDF5 format (not with pickle)
model_filename = 'retinal_heart_risk_model.h5'
model.save(model_filename)

# 7. Individual evaluation for each input image
for i in range(len(X_test)):
    print(f"\nImage {i + 1}:")
    y_true = y_test[i]
    y_pred_single = model.predict(np.expand_dims(X_test[i], axis=0))[0]

    print(f"True Values: {y_true}")
    print(f"Predicted Values: {y_pred_single}")

    # Calculate MAE for each prediction
    mae_single = np.mean(np.abs(y_true - y_pred_single))
    print(f"Mean Absolute Error (MAE): {mae_single}")



R2 Score: -1.4718930393386193e+27
Mean Absolute Error (MAE): [3.41770962 4.58692139 2.05123919 1.21586131 0.85781305]
Mean Squared Error (MSE): [13.47980463 30.42605419  7.03412181  1.82921025  0.83600559]
Suggested Precautions:  ['Monitor blood sugar levels and consult a doctor.']

Image 1:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
True Values: [ 45.  130.   80.   26.5   6.2]
Predicted Values: [ 47.22417   130.56236    78.76048    24.610014    6.9242234]
Mean Absolute Error (MAE): 1.3280520057678222

Image 2:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
True Values: [ 45.  130.   80.   26.5   6.2]
Predicted Values: [ 46.614197 128.07176   77.26243   24.618666   6.776851]
Mean Absolute Error (MAE): 1.7476383781433107

Image 3:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
True Values: [ 45.  130.   80.   26.5   6.2]
Predicted Values: [ 48.869183 134.82019   80.93054   25.569183   6.894789]
Mean Absolute Error (MAE): 2.249104175567627

Image 4:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
True Values: [ 45.  13

In [ ]:

# Directories for training and test data
# train_dir = '/content/retinal_disease_classification_data/Training_Set/Training_Set/Training'
# test_dir = '/content/retinal_disease_classification_data/Test_Set/Test_Set/Test'

# train_dir = '/content/retinal_disease_classification_data/Evaluation_Set/Evaluation_Set/Validation'
# test_dir = '/content/retinal_disease_classification_data/Evaluation_Set/Evaluation_Set/Validation'

# train_dir =  '/content/retinal_disease_classification_data/Training_Set/Training_Set/Training'
# val_dir = '/content/retinal_disease_classification_data/Evaluation_Set/Evaluation_Set/Validation'
# test_dir =  '/content/retinal_disease_classification_data/Test_Set/Test_Set/Test'

In [ ]:
!pip install timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 25.5 MB/s eta 0:00:00


In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Specify the directory containing images
image_dir = '/content/retinal_disease_classification_data/Evaluation_Set/Evaluation_Set/Validation/'

# List all files in the directory
image_files = os.listdir(image_dir)

# Initialize lists to hold images and corresponding health data
X = []
y = []

# Process each image
for img_file in image_files:
    image_path = os.path.join(image_dir, img_file)

    # Load the retinal image
    image = load_img(image_path, target_size=(224, 224))  # Resize each image
    image = img_to_array(image)
    image = image / 255.0  # Normalize pixel values

    # Append the image to the X list
    X.append(image)

    # Simulate health labels: [age, sbp, dbp, bmi, hba1c]
    # In practice, you would have these values from your dataset
    y.append([45, 130, 80, 26.5, 6.2])  # Example target labels

# Convert lists to numpy arrays
X = np.array(X)
y = np.array(y)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 2. Build the CNN model for feature extraction and prediction
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(5, activation='linear')  # Output: [age, sbp, dbp, bmi, hba1c]
])

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# 3. Train the model
model.fit(X_train, y_train, epochs=10, validation_split=0.2, verbose=1)

# 4. Make predictions
y_pred = model.predict(X_test)

# Evaluate the model: R2 score and MAE
r2 = r2_score(y_test, y_pred)
print(f"R2 Score: {r2}")

# 5. Suggest health precautions based on the predictions
def suggest_precautions(y_pred):
    age, sbp, dbp, bmi, hba1c = y_pred[0]
    suggestions = []
    if sbp > 140 or dbp > 90:
        suggestions.append("Monitor blood pressure regularly and reduce salt intake.")
    if bmi > 25:
        suggestions.append("Maintain a healthy diet and engage in regular exercise.")
    if hba1c > 6.5:
        suggestions.append("Monitor blood sugar levels and consult a doctor.")
    if age > 50:
        suggestions.append("Consider regular heart check-ups.")
    return suggestions

# Generate suggestions based on predictions
precautions = suggest_precautions(y_pred)
print("Suggested Precautions: ", precautions)

# 6. Save the trained model in HDF5 format (not with pickle)
model_filename = 'retinal_heart_risk_model.h5'
model.save(model_filename)


Epoch 1/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 49s 4s/step - loss: 3268.2881 - mae: 43.2396 - val_loss: 736.9901 - val_mae: 21.0250
Epoch 2/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 46s 4s/step - loss: 770.5345 - mae: 21.5994 - val_loss: 372.0696 - val_mae: 14.4978
Epoch 3/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 84s 4s/step - loss: 559.8062 - mae: 18.1145 - val_loss: 223.7213 - val_mae: 10.3513
Epoch 4/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 84s 4s/step - loss: 491.3237 - mae: 16.7900 - val_loss: 113.4465 - val_mae: 6.9547
Epoch 5/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 78s 4s/step - loss: 494.4073 - mae: 16.8395 - val_loss: 75.7691 - val_mae: 5.3548
Epoch 6/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 46s 4s/step - loss: 427.3336 - mae: 15.8304 - val_loss: 90.1678 - val_mae: 6.2715
Epoch 7/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 82s 4s/step - loss: 309.1921 - mae: 13.4610 - val_loss: 27.4667 - val_mae: 3.4095
Epoch 8/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 82s 3s/step - loss: 300.8399 - mae: 13.4809 - val_loss: 11.9250 - val_mae: 2.0858
Epoch 9/10
13/13 ━━━━━━━━━━━━━━━

R2 Score: -1.0333436667303528e+27
Suggested Precautions:  []


In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Load the trained model
model_filename = 'retinal_heart_risk_model.h5'
model = load_model(model_filename)

# Specify the directory containing new images for testing
test_image_dir = '/content/retinal_disease_classification_data/Test_Set/Test_Set/Test'  # Adjust this path as needed

# List all files in the directory
test_image_files = os.listdir(test_image_dir)

# Initialize list to hold test images
X_test_new = []

# Process each test image
for img_file in test_image_files:
    image_path = os.path.join(test_image_dir, img_file)

    # Load and preprocess the image
    image = load_img(image_path, target_size=(224, 224))  # Resize each image
    image = img_to_array(image)
    image = image / 255.0  # Normalize pixel values

    # Append the image to the test list
    X_test_new.append(image)

# Convert list to numpy array
X_test_new = np.array(X_test_new)

# Make predictions on the new test images
y_pred_new = model.predict(X_test_new)

# Print predictions for each test image
for i, img_file in enumerate(test_image_files):
    print(f"Predictions for {img_file}: Age: {y_pred_new[i][0]}, SBP: {y_pred_new[i][1]}, DBP: {y_pred_new[i][2]}, BMI: {y_pred_new[i][3]}, HbA1c: {y_pred_new[i][4]}")


20/20 ━━━━━━━━━━━━━━━━━━━━ 19s 946ms/step
Predictions for 496.png: Age: 39.90604782104492, SBP: 112.86878204345703, DBP: 71.41351318359375, BMI: 23.858192443847656, HbA1c: 5.266858100891113
Predictions for 326.png: Age: 39.983890533447266, SBP: 112.86055755615234, DBP: 70.68494415283203, BMI: 23.87523651123047, HbA1c: 5.312103748321533
Predictions for 468.png: Age: 39.96013259887695, SBP: 112.73912811279297, DBP: 71.3984375, BMI: 23.871437072753906, HbA1c: 5.452840805053711
Predictions for 444.png: Age: 39.63027572631836, SBP: 112.29586029052734, DBP: 70.9380874633789, BMI: 23.809223175048828, HbA1c: 5.113999843597412
Predictions for 141.png: Age: 41.30209732055664, SBP: 116.24468994140625, DBP: 73.06050109863281, BMI: 24.438251495361328, HbA1c: 5.554649829864502
Predictions for 311.png: Age: 39.943660736083984, SBP: 112.5084228515625, DBP: 70.6986312866211, BMI: 23.71441650390625, HbA1c: 5.398900032043457
Predictions for 637.png: Age: 41.67329788208008, SBP: 118.06303405761719, DBP: 7